# MCMC6.0: Hubbard Model

The algorithm of quantum Monte Carlo (QMC) simulations for the Hubbard model is one of the greatest human wisdoms in physics.

## Lazy evaluation

Before going on to the real implementation of the Hubbard model QMC, I want to explain something not realted to the Hubbard model. Lazy evaluation is one of the most important techniques in the functional programming, but it is not completely installed in Julia 1.0. I have simulated lazy evaluation or infinite lists by using iterators and generators, but the functionality is still limited.

As I discussed in MCMC0.5, map in Julia does not do lazy evaluation, so I define a lazy version.

In [1]:
lazymap = Base.Generator

Base.Generator

In [2]:
collect(lazymap(+, 1 : 5, 2 : 6))

5-element Array{Int64,1}:
  3
  5
  7
  9
 11

From now on, we use the following statistical operation instead of Statistics.

In [3]:
function meanstd(itr)::Tuple
    moment = sum(lazymap(x -> [x, x ^ 2], itr))
    n = length(itr)
    mean = moment[1] / n
    mean, sqrt((moment[2] - (mean ^ 2) * n) / (n - 1))
end

meanstd (generic function with 1 method)

In [4]:
meanstd(1 : 5)

(3.0, 1.5811388300841898)

In this way, we can completely avoid storing every sample into the memory. Jackknife methods can be rewritten as follows.

In [5]:
using ResumableFunctions
@resumable function deleteone(itr::AbstractVector{T})::Vector{T} where T
    n = length(itr)
    k, old = iterate(itr)
    new = old # for the type stability
    for i in 1 : (n - 1)
        k, new = iterate(itr, old)
        @yield [ifelse(j <= i, new, old) for j in 1 : n]
        old = new
    end
end
collect(deleteone(1 : 5))

4-element Array{Array{Int64,1},1}:
 [2, 1, 1, 1, 1]
 [3, 3, 2, 2, 2]
 [4, 4, 4, 3, 3]
 [5, 5, 5, 5, 4]

This is a very bad code... I think there is a better way using Iterators.filter. Anyway, this works enough well for our purpose. Until this function (deleteone), you should only use lazy functions like lazymap. In this way, the observables will be calculated only when you calculate the mean values for each jackknifed sample. Jackknife means have to be calculated directly from the vector, so you still need an $O(N)$ memory space.

In [6]:
using Statistics
leaveoneout(before::Function, after::Function, itr::AbstractVector) = map(after, mean(deleteone(lazymap(before, itr))))
function jackknifemeanstd(b::Function, a::Function, itr::AbstractVector)::Tuple
    v = leaveoneout(b, a, itr)
    m = mean(v)
    s = stdm(v, m, corrected = false) * sqrt(length(itr) - 1)
    m, s
end

jackknifemeanstd (generic function with 1 method)

In the Kitaev model, lazy evaluation was not so important because there is no reason to store any matrices. However, in the Hubbard model, each state keeps a Green function as an observable, so it is not memory-efficient to yield all the matrices (Green functions) before the statistical operations. In this case, using lazy evaluation must be important.